In [1]:
!nvidia-smi

Sun May  8 16:09:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:3B:00.0 Off |                  Off |
| 32%   43C    P0    67W / 260W |      0MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     Off  | 00000000:AF:00.0 Off |                  Off |
| 32%   

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import torch
import numpy as np

torch.__version__

'1.10.1+cu102'

In [4]:
np_arr_1 = np.array([1,2,3,4])
np_arr_2 = np.array([5,6,7,8])

torch_tensor_1 = torch.tensor([1,2,3,4])
torch_tensor_2 = torch.tensor([5,6,7,8])

torch_tensor_1

tensor([1, 2, 3, 4])

In [5]:
print(np_arr_1.shape)

print(torch_tensor_1.shape)
print(torch_tensor_1.size())

(4,)
torch.Size([4])
torch.Size([4])


In [6]:
np_concat = np.concatenate([np_arr_1, np_arr_2], axis=0)
print('------numpy-------')
print(np_concat)

torch_concat = torch.cat([torch_tensor_1, torch_tensor_2], dim=0)
print('------torch-------')
print(torch_concat)

------numpy-------
[1 2 3 4 5 6 7 8]
------torch-------
tensor([1, 2, 3, 4, 5, 6, 7, 8])


In [7]:
# repeat 기능

x = np.array([1,2,3])
x_repeat = x.repeat(2)

print('-------numpy-------')
print(x)
print(x_repeat)

y = torch.tensor([1,2,3])
y_repeat = y.repeat(2)

print('-------tensor--------')
print(y)
print(y_repeat)

-------numpy-------
[1 2 3]
[1 1 2 2 3 3]
-------tensor--------
tensor([1, 2, 3])
tensor([1, 2, 3, 1, 2, 3])


In [8]:
# is cuda(GPU) available?
torch.cuda.is_available()

True

In [34]:
a = torch.ones(3)
b = torch.rand(100, 10, 3)

print(a.device)
print(b.device)

print(a.device)

cpu
cpu
cpu


In [10]:
print(b[0])
print(b[0][0])      # same as b[0,0]
print(b[0][0][0])   # same as b[0,0,0]

tensor([[0.9303, 0.9332, 0.6412],
        [0.3405, 0.2436, 0.1521],
        [0.6839, 0.4895, 0.1862],
        [0.1234, 0.3155, 0.8768],
        [0.2277, 0.2525, 0.3966],
        [0.1587, 0.6268, 0.8978],
        [0.3108, 0.0575, 0.5294],
        [0.7722, 0.8618, 0.9845],
        [0.9007, 0.8824, 0.1666],
        [0.2748, 0.8925, 0.3795]])
tensor([0.9303, 0.9332, 0.6412])
tensor(0.9303)


In [11]:
len(b)

100

In [12]:
# Autograd
# automatically defferentiate tensors

x = torch.ones(2,2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [13]:
y = x+2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [14]:
z = y*y+2
z

tensor([[11., 11.],
        [11., 11.]], grad_fn=<AddBackward0>)

In [15]:
out = z.mean()
out

tensor(11., grad_fn=<MeanBackward0>)

In [16]:
y.retain_grad()
z.retain_grad()
out.backward()

In [17]:
z.grad

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])

In [18]:
y.grad

tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])

In [20]:
# computer remembers gradients of previous calculations
# # so we sould use .no_grad() to prevent complexity

In [26]:
import torch.nn as nn

x = torch.tensor([[1.,2.,3.], [4.,5.,6.]])
x

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [27]:
linear_fn = nn.Linear(3,1)
linear_fn

Linear(in_features=3, out_features=1, bias=True)

In [30]:
y = linear_fn(x)
print(y)
print(y.shape)

tensor([[1.6417],
        [2.7969]], grad_fn=<AddmmBackward0>)
torch.Size([2, 1])


In [31]:
y.sum()

tensor(4.4386, grad_fn=<SumBackward0>)

In [32]:
# Design model using Pytorch

class Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)        # activation function
        x = self.linear2(x)
        return x

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [37]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

# DataLoader
# mini batch size
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [38]:
# Define model class

class Multimodal_logistic_regression(nn.Module):
    def __init__(self, input_size, output_size):
        super(Multimodal_logistic_regression, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.fc(x)
        return out

In [41]:
# model generation

model = Multimodal_logistic_regression(784, 10)
model = model.to('cuda')
model

Multimodal_logistic_regression(
  (fc): Linear(in_features=784, out_features=10, bias=True)
)

In [44]:
# Define optimizer

optim_sgd = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
optim_adam = torch.optim.Adam(model.parameters(), lr=0.05)

In [47]:
# model training

loss_fn = nn.CrossEntropyLoss()

total_step = len(train_loader)

for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to('cuda')
        labels = labels.to('cuda')
        # print(images.shape)

        # forward
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # backward and optimization
        optimizer.zero_grad()
        loss.backward()     # automatic gradient calculation (autograd)
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch [{epoch+1}/{10}], Loss: {loss.item():.4f}')

epoch [1/10], Loss: 0.2661
epoch [1/10], Loss: 0.3099
epoch [1/10], Loss: 0.1870
epoch [1/10], Loss: 0.3270
epoch [2/10], Loss: 0.2287
epoch [2/10], Loss: 0.2344
epoch [2/10], Loss: 0.2785
epoch [2/10], Loss: 0.3494
epoch [3/10], Loss: 0.2830
epoch [3/10], Loss: 0.4959
epoch [3/10], Loss: 0.2351
epoch [3/10], Loss: 0.2606
epoch [4/10], Loss: 0.2717
epoch [4/10], Loss: 0.2498
epoch [4/10], Loss: 0.1603
epoch [4/10], Loss: 0.1783
epoch [5/10], Loss: 0.1531
epoch [5/10], Loss: 0.2677
epoch [5/10], Loss: 0.3527
epoch [5/10], Loss: 0.2834
epoch [6/10], Loss: 0.3579
epoch [6/10], Loss: 0.2871
epoch [6/10], Loss: 0.1906
epoch [6/10], Loss: 0.3247
epoch [7/10], Loss: 0.1637
epoch [7/10], Loss: 0.2255
epoch [7/10], Loss: 0.2870
epoch [7/10], Loss: 0.3344
epoch [8/10], Loss: 0.1856
epoch [8/10], Loss: 0.2927
epoch [8/10], Loss: 0.2666
epoch [8/10], Loss: 0.2546
epoch [9/10], Loss: 0.2372
epoch [9/10], Loss: 0.2813
epoch [9/10], Loss: 0.2094
epoch [9/10], Loss: 0.1717
epoch [10/10], Loss: 0.2469


In [50]:
# model testing

with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to('cuda')
        labels = labels.to('cuda')
        outputs = model(images)

        _, predictions = torch.max(outputs.data, 1)
        print(predictions)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

    print(f'Acc: {100 * correct/total}')

tensor([0, 4, 3, 4, 4, 6, 3, 7, 6, 4, 7, 4, 8, 5, 1, 0, 5, 0, 8, 3, 5, 3, 4, 3,
        0, 5, 1, 1, 5, 6, 5, 9, 0, 7, 2, 3, 8, 9, 0, 9, 2, 7, 4, 2, 2, 2, 1, 7,
        8, 5, 9, 9, 2, 0, 3, 0, 8, 9, 7, 0, 1, 4, 1, 3, 8, 1, 9, 2, 1, 1, 9, 5,
        9, 5, 7, 4, 1, 7, 2, 0, 1, 3, 3, 3, 3, 6, 4, 1, 9, 9, 4, 7, 2, 7, 1, 2,
        3, 9, 3, 7], device='cuda:0')
tensor([9, 3, 1, 5, 4, 5, 7, 8, 4, 4, 4, 6, 3, 4, 3, 7, 1, 4, 0, 6, 8, 1, 0, 5,
        6, 6, 6, 5, 0, 4, 7, 1, 6, 2, 8, 6, 7, 6, 9, 1, 3, 8, 5, 5, 6, 1, 0, 9,
        0, 4, 6, 1, 0, 1, 1, 7, 8, 4, 1, 2, 6, 2, 6, 5, 0, 9, 4, 7, 8, 0, 7, 0,
        1, 1, 1, 0, 6, 1, 2, 7, 0, 0, 5, 7, 7, 0, 9, 7, 5, 1, 9, 3, 3, 3, 5, 2,
        2, 8, 9, 0], device='cuda:0')
tensor([6, 6, 3, 2, 6, 7, 0, 4, 0, 3, 0, 3, 9, 5, 7, 5, 1, 6, 5, 8, 6, 0, 3, 2,
        1, 4, 6, 1, 8, 3, 3, 9, 1, 0, 0, 2, 9, 5, 3, 8, 2, 0, 1, 7, 5, 8, 4, 1,
        7, 1, 2, 1, 6, 4, 8, 4, 0, 7, 1, 1, 0, 3, 8, 1, 3, 9, 8, 7, 3, 7, 1, 1,
        3, 9, 5, 7, 8, 3, 6, 3, 1, 3, 3, 7, 